### LangChain RAG 파헤치기: 문서 기반 QA 시스템 설계 방법 - 심화편

- 1. 도큐먼트 로더(document_loader): RAW DATA 로부터 데이터 읽기
- 2. 문서 분할(splitter): 문서를 규칙에 의하여 Chunk 로 분할
- 3. Embedding: 다양한 임베딩 알고리즘
- 4. VectorStore: 벡터DB 에 데이터 저장
- 5. Embedding: 데이터 저장시 사용할 임베더 선택
- 6. Retriever: 사용자 쿼리에 따른 데이터 검색
- 7. 프롬프트(Prompt): LangSmith 의 프롬프트 로드
- 8. 모델(LLM): OpenAI, HuggingFace 오픈소스 모델
- 9. 체인 생성: 체인 생성 후 쿼리(query)
- 10. RAG 템플릿 실험: 실제 문서를 활용한 실험 결과 공유 

In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key  = os.environ['OPENAI_API_KEY']

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

1. 도큐먼트 로더(document_loader): RAW DATA 로부터 데이터 읽기
- 참조: https://python.langchain.com/docs/integrations/document_loaders/

In [ ]:
# 웹페이지
# 뉴스기사의 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/news/business-68092814",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "main",
            attrs={"id": ["main-content"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
print(docs[0].page_content[:500])

In [7]:
# PDF
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("./SPRI_AI_Brief_2023년12월호_F.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 23

[페이지내용]
SPRi AI Brief |  
2023-12 월호
8코히어 , 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n코히어와 12개 기관이  광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기 ’ 플랫폼을 출시
n대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능KEY Contents
£데이터 출처 탐색기 , 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
nAI 기업 코히어 (Cohere) 가 매사추세츠 공과⼤(MIT), 하버드 ⼤ 로스쿨 , 카네기멜론 ⼤ 등 12개 기관과  
함께 2023 년 10월 25일 ‘데이터 출처 탐색기 (Data Provenance Explorer)’ 플랫폼을 공개
∙AI 모델 훈련에 사용되는 데이터셋의 불분명한 출처로 인해 데이터 투명성이 확보되지 않아 다양한 
법적·윤리적 문제가 발생
∙이에 연구진은 가장 널리 사용되는 2,000 여 개의 미세조정 데이터셋을 감사 및 추적하여 데이터셋에 
원본 데이터소스에 대한 태그, 재라이선스 (Relicensing) 상태, 작성자 , 기타 데이터 속성을 지정하고 
이러한 정보에 접근할 수 있는 플랫폼을 출시
∙대화형 플랫폼 형태의 데이터 출처 탐색기를 통해 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 , 
주요 데이터셋의 구성과 데이터 계보도 추적 가능
n연구진은 오픈소스 데이터셋에 대한 광범위한 감사를 통해 데이터 투명성에 영향을 미치는 주요 
요인을  발견
∙깃허브 (GitHub), 페이퍼위드코드 (Papers with Code) 와 같은 크라우드소싱 플랫폼에서 수집한 
데이터로 훈련된 오픈소스 LLM에서는 데이터 라이선스의 누락 비율이 72~83% 에 달함 
∙또한 크라우드소싱 플랫폼이 할당한 라이선스는 데이터셋 원저작자의 의도보다 더 광범위한 사용을 
허용한 경우가 상당수
∙데이터 생태계 분

In [ ]:
# CSV
# CSV 파일 로드
loader = CSVLoader(file_path="data/titanic.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

 2. 문서 분할(splitter): 문서를 규칙에 의하여 Chunk 로 분할
- 3. Embedding: 다양한 임베딩 알고리즘
- 4. VectorStore: 벡터DB 에 데이터 저장
- 5. Embedding: 데이터 저장시 사용할 임베더 선택
- 6. Retriever: 사용자 쿼리에 따른 데이터 검색
- 7. 프롬프트(Prompt): LangSmith 의 프롬프트 로드
- 8. 모델(LLM): OpenAI, HuggingFace 오픈소스 모델
- 9. 체인 생성: 체인 생성 후 쿼리(query)
- 10. RAG 템플릿 실험: 실제 문서를 활용한 실험 결과 공유 

In [ ]:
# <이전 단계에서 수행 했던 것>

# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/article/437/0000378416"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "부영그룹의 출산 장려 정책에 대해 설명해주세요"
response = rag_chain.invoke(question)

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")